In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
import nltk

import json
import csv

import plotly.graph_objects as go

import string
import sys
import random

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [2]:
import os
if os.getcwd().split('/')[-1] != 'winogrande_1.1':
    os.chdir('/home/bssachde/Winogrande/winogrande_1.1')

In [3]:
data = []
with open('train.jsonl') as f:
    for line in f:
        data.append(json.loads(line))

In [4]:
D=pd.DataFrame(data)
S=D

In [5]:
S

,qID,sentence,option1,option2,answer
0,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-2,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,2
1,3QHITW7OYO7Q6B6ISU2UMJB84ZLAQE-1,Ian volunteered to eat Dennis's menudo after a...,Ian,Dennis,1
2,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-1,"He never comes to my home, but I always go to ...",home,house,1
3,3XWUWJ18TLO2DDRXF83QWLKRJ29UU4-2,"He never comes to my home, but I always go to ...",home,house,2
4,3D5G8J4N5CI2K40F4RZLF9OG2CKVTH-2,"Kyle doesn't wear leg warmers to bed, while Lo...",Kyle,Logan,2
...,...,...,...,...,...
9995,3ZG552ORAM2T6G7V1E3PMORI94N2VA-1,Studying economics was far easier for Megan th...,Megan,Monica,1
9996,3ZLW647WAN9OOW4J129JVRN8D4F32J-1,Lorelei took the carnation out of the jar and ...,jar,vase,1
9997,3ZLW647WAN9OOW4J129JVRN8D4F32J-2,Lorelei took the carnation out of the jar and ...,jar,vase,2
9998,3KA7IJSNW63AP9AVYVN1HP54HO0BP5-2,Felicia only likes dogs for pets while Laura o...,Felicia,Laura,2


In [6]:
n = 5000 # Target Dataset

In [7]:
## Load module for test train split
from sklearn.model_selection import train_test_split

## Load set of Linear models
from sklearn.linear_model import LogisticRegression

## Load non-Linear Models
from sklearn import svm

In [8]:
# transformer libraries for fine tune embeddings
from pytorch_transformers import BertPreTrainedModel, RobertaConfig, \
    ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP, RobertaModel
from pytorch_transformers.modeling_roberta import RobertaClassificationHead
from torch.nn import CrossEntropyLoss



In [9]:
## This is for pretraining
# !git clone https://github.com/allenai/winogrande.git

In [10]:
# # import os
# # os.chdir("winogrande/")
# !export PYTHONPATH=$PYTHONPATH:$(pwd)
# !python "scripts/run_experiment.py" --model_type roberta_mc --model_name_or_path roberta-large --task_name winogrande --do_eval --do_lower_case --data_dir ./data  --max_seq_length 80  --per_gpu_eval_batch_size 4  --per_gpu_train_batch_size 16  --learning_rate 1e-5  --num_train_epochs 3  --output_dir ./output/models/  --do_train  --logging_steps 4752  --save_steps 4750  --seed 42  --data_cache_dir ./output/cache/  --warmup_pct 0.1  --evaluate_during_training

In [11]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)#, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

In [12]:
output_model_file = "models/pytorch_model.bin"
model_state_dict = torch.load(output_model_file) 
model = RobertaModel.from_pretrained('roberta-large',state_dict=model_state_dict)

In [13]:
outputs = model(input_ids)#, masked_lm_labels=input_ids)

In [14]:
# # sentences = ['This framework generates embeddings for each input sentence',
# #     'Sentences are passed as a list of string.']
# # outputembeddings = model.get_output_embeddings()
# X = []
# len(S['sentence'])
# for sentence in S['sentence']:
# #     print(sentence)
#     input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#     X.append(model(input_ids)[0])#, masked_lm_labels=input_ids)

In [27]:
import pickle
import os

X_prime = []
## Check if there was already some progress then continue from there otherwise just start from begining
if not os.path.isfile("start.data"):
    start = 0
else:
    with open("start.data","r") as fileHandle:
        start = fileHandle.readline()
        # Scenario if file is there but nothing is in the file
        if start == '':
            start = 0
for index in range(int(start),len(S['sentence'])):
    if index%100==0:
        print("Saving the progress for embeddings")
        with open('listfile.data', 'ab') as filehandle:
            pickle.dump(X_prime, filehandle)
        ## Reset the embeddings list X_prime and just keep appending the data in the file
        X_prime = []
        ## overwrite the index so far in seprate file
        with open("start.data","w") as fileHandle:
            fileHandle.write(str(index))
    print(index)
    input_ids = torch.tensor(tokenizer.encode(S['sentence'][index], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    X_prime.append(model(input_ids)[0])#, masked_lm_labels=input_ids)

Saving the progress for embeddings


KeyboardInterrupt: 

In [ ]:
# If it fails again here just read the file here and run again

In [ ]:
outputs[0].shape

In [ ]:
# while len(S) > n:
#Dictionary to save the mdoel predictions for every individual elements.
E = dict.fromkeys(S['qID'])
EPrime = dict.fromkeys(S['sentence'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
## Train models
# lm = LogisticRegression(random_state=0)
# lm.fit(X_train, y_train)
# rbf_svc = svm.SVC(kernel='rbf')
# rbf_svc.fit(X_train,y_train)
## Predictions
# predictions = lm.predict(X_test)
# predictions = rbf_svc.predict(X_test)